# Vehicle Loan Default

In [1]:
# !pip install numpy
# !pip install pandas
# !pip install scikit-learn
# !pip install dask
# !pip install matplotlib
# !pip install statsmodels
# !pip install seaborn
# !pip install sklearn

# Load Packages

In [31]:
import numpy as np
import pandas as pd
import re
import datetime as dt
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import os
import random
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score

random.seed(82122)

%matplotlib inline

# Read in Data

In [3]:
cwd = os.getcwd()

inputdir = "C:\\devl\\VehicleLoanDefault\\data"
outputdir = "C:\\devl\\VehicleLoanDefault\\output"

dat = pd.read_csv(inputdir + "\\train.csv", na_values = '?') #, nrows = 10000)
dat.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,...,0,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,...,0,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1


In [4]:
print(dat.shape)
print(dat.dtypes)


(233154, 41)
UniqueID                                 int64
disbursed_amount                         int64
asset_cost                               int64
ltv                                    float64
branch_id                                int64
supplier_id                              int64
manufacturer_id                          int64
Current_pincode_ID                       int64
Date.of.Birth                           object
Employment.Type                         object
DisbursalDate                           object
State_ID                                 int64
Employee_code_ID                         int64
MobileNo_Avl_Flag                        int64
Aadhar_flag                              int64
PAN_flag                                 int64
VoterID_flag                             int64
Driving_flag                             int64
Passport_flag                            int64
PERFORM_CNS.SCORE                        int64
PERFORM_CNS.SCORE.DESCRIPTION           object


In [5]:
dat['loan_default'].mean()

0.2170711203753742

### Drop seconary loan columns for now

In [6]:
dat = dat.drop(columns = [cols for cols in dat.columns if "SEC." in cols])

### Convert ID Columns to strings, drop PIN codes and employee codes

In [16]:
id_cols = [c for c in dat.columns if re.compile('id', re.IGNORECASE).search(c)]
id_cols.remove('VoterID_flag')
print(id_cols)


['UniqueID', 'branch_id', 'supplier_id', 'manufacturer_id', 'Current_pincode_ID', 'State_ID', 'Employee_code_ID']


In [18]:
dat[id_cols] = dat[id_cols].astype(str)

In [20]:
dat.drop(columns = ['Current_pincode_ID', 'Employee_code_ID'])

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Date.of.Birth,Employment.Type,DisbursalDate,...,PRI.CURRENT.BALANCE,PRI.SANCTIONED.AMOUNT,PRI.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default
0,420825,50578,58400,89.55,67,22807,45,01-01-84,Salaried,03-08-18,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
1,537409,47145,65550,73.23,67,22807,45,31-07-85,Self employed,26-09-18,...,27600,50200,50200,1991,0,1,1yrs 11mon,1yrs 11mon,0,1
2,417566,53278,61360,89.63,67,22807,45,24-08-85,Self employed,01-08-18,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
3,624493,57513,66113,88.48,67,22807,45,30-12-93,Self employed,26-10-18,...,0,0,0,31,0,0,0yrs 8mon,1yrs 3mon,1,1
4,539055,52378,60300,88.39,67,22807,45,09-12-77,Self employed,26-09-18,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233149,626432,63213,105405,60.72,34,20700,48,01-08-88,Salaried,26-10-18,...,390443,416133,416133,4084,0,0,1yrs 9mon,3yrs 3mon,0,0
233150,606141,73651,100600,74.95,34,23775,51,05-12-88,Self employed,23-10-18,...,0,0,0,1565,0,0,0yrs 6mon,0yrs 6mon,0,0
233151,613658,33484,71212,48.45,77,22186,86,01-06-76,Salaried,24-10-18,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0
233152,548084,34259,73286,49.10,77,22186,86,26-03-94,Salaried,29-09-18,...,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0


### Convert Flags to binary

In [92]:
dat[dat['Date.of.Birth'].str.len() < 4 , 'Date.of.Birth']

InvalidIndexError: (0         False
1         False
2         False
3         False
4         False
          ...  
233149    False
233150    False
233151    False
233152    False
233153    False
Name: Date.of.Birth, Length: 233154, dtype: bool, 'Date.of.Birth')

In [78]:
pd.isnull(dat['Date.of.Birth']).sum()

0

In [125]:
def datestring_to_age(datestring, fmt = '%d-%m-%y'):
    try:
        thing = dt.datetime.strptime(datestring, fmt)
        return dt.datetime.today() - thing
    except ValueError:
        print("not cool!" + datestring)


In [129]:
[datestring_to_age(x).days for x in dat['Date.of.Birth']]

[14112,
 13535,
 13511,
 10461,
 16326,
 11670,
 12499,
 12009,
 11237,
 -16721,
 15938,
 13862,
 17613,
 13884,
 18081,
 10274,
 16882,
 14420,
 12429,
 15467,
 15999,
 14053,
 10265,
 15788,
 12284,
 13381,
 -16783,
 17034,
 18240,
 19413,
 17764,
 11778,
 -15990,
 9519,
 16517,
 10547,
 13117,
 11555,
 16499,
 9501,
 19439,
 10787,
 14672,
 11769,
 16703,
 11857,
 10810,
 13230,
 17681,
 15316,
 13381,
 18746,
 12134,
 10305,
 13894,
 13850,
 10094,
 10646,
 17034,
 -15260,
 16873,
 16059,
 14112,
 18495,
 10094,
 13746,
 13746,
 11555,
 -16204,
 13746,
 14477,
 10094,
 -16204,
 15573,
 10824,
 12526,
 18129,
 13381,
 14427,
 13381,
 19225,
 19074,
 10746,
 16303,
 19439,
 9363,
 16303,
 8782,
 8579,
 14477,
 10267,
 8668,
 15938,
 10824,
 17243,
 14467,
 16303,
 -16569,
 11822,
 8944,
 16668,
 11920,
 10681,
 15480,
 16303,
 12285,
 15938,
 17034,
 14477,
 11920,
 16303,
 -16204,
 12328,
 10459,
 11555,
 12285,
 19590,
 18495,
 15308,
 15207,
 9970,
 12651,
 13016,
 13960,
 16668,


In [130]:
dat['Date.of.Birth'].head(15)

0     01-01-84
1     31-07-85
2     24-08-85
3     30-12-93
4     09-12-77
5     08-09-90
6     01-06-88
7     04-10-89
8     15-11-91
9     01-06-68
10    01-01-79
11    07-09-84
12    01-06-74
13    16-08-84
14    18-02-73
Name: Date.of.Birth, dtype: object

In [76]:


# Convert DOB to age

# pd.to_datetime(dat['Date.of.Birth'])

dat['Date.of.Birth'] = [dt.datetime.strptime(x, '%y-%m-%d') for x in dat['Date.of.Birth']]
dat['Date.of.Birth'].head()
# now = dt.date.today()
# dat['Age'] = dat['Date.of.Birth'].apply(lambda x: now - dt.datetime.strptime(x, '%y-%m-%d'))

# Convert Disbursal Date to Loan age
# Investigate CNS score and description
# Investigate numbers of accounts - possibly convert to categorical
# Get ratios sanctioned:disbursed, (disbursed - current balance):current balance, 
# install amount:disbursed, install amount:current balance

ValueError: unconverted data remains: 4

In [68]:
dat['Age'].head()

KeyError: 'Age'

In [7]:
[cols for cols in dat.columns if dat[cols].dtype == np.int64]

['UniqueID',
 'disbursed_amount',
 'asset_cost',
 'branch_id',
 'supplier_id',
 'manufacturer_id',
 'Current_pincode_ID',
 'State_ID',
 'Employee_code_ID',
 'MobileNo_Avl_Flag',
 'Aadhar_flag',
 'PAN_flag',
 'VoterID_flag',
 'Driving_flag',
 'Passport_flag',
 'PERFORM_CNS.SCORE',
 'PRI.NO.OF.ACCTS',
 'PRI.ACTIVE.ACCTS',
 'PRI.OVERDUE.ACCTS',
 'PRI.CURRENT.BALANCE',
 'PRI.SANCTIONED.AMOUNT',
 'PRI.DISBURSED.AMOUNT',
 'PRIMARY.INSTAL.AMT',
 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
 'NO.OF_INQUIRIES',
 'loan_default']